## Optimizing neural network parameters

In [1]:
import util
import platform
from pathlib import Path
from os import path

import numpy as np
from scipy.io import loadmat

import yass
from yass.augment import make
from yass.neuralnetwork import NeuralNetDetector, NeuralNetTriage

In [2]:
LOCAL = platform.system() == 'Darwin'

if LOCAL:
    yass.set_config("../config/49-local.yaml")
else:
    yass.set_config("../config/49-lab.yaml")

# yass configuration
home = Path.home()
path_to_config = str(home / 'dev/private-yass/config/49-lab.yaml')
path_to_ground_truth = str(home / 'data/groundtruth_ej49_data1_set1.mat')
path_to_data = str(home / 'data/tmp')

yass.set_config(path_to_config)
CONFIG = yass.read_config()

if LOCAL:
    path_to_here = path.expanduser('~/dev/lab/private-yass/nnet')
else:
    path_to_here = path.expanduser('~/dev/private-yass/nnet')

In [3]:
# load ground truth
_ = loadmat(path_to_ground_truth)
spike_train = np.hstack([_['spt_gt'], _['L_gt']])

spike_train = spike_train[2:-1]
spike_train[:, 1] = spike_train[:, 1] - 1

# compensate alignment
spike_train[:, 0] = spike_train[:, 0] + 10

In [4]:
import itertools

# training set parameters

n_spikes = [20000]
min_amplitude = [5]
max_amplitude = [90]

n_templates = np.max(spike_train[:,1]) + 1
chosen_templates = [np.arange(n_templates)]

# chosen_templates = [0, 1, 2, 3, 4, 5, 7, 8, 9, 11, 12, 13,
#                     14, 15, 16, 17, 18, 19, 22, 23, 24, 25,
#                     26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
#                     36, 37, 38, 39, 41, 42, 43, 44, 45, 46,
#                     47, 48]

set_parameters = itertools.product(n_spikes, min_amplitude,
                                   max_amplitude, chosen_templates)

# model parameters
n_iter = 5000
l2_reg_scale = [0.00000005, 0.0000005, 0.000005]
filters_size = [[32, 16]]

model_parameters = filters_size


parameters = itertools.product(set_parameters, model_parameters)

In [5]:
for (n_spikes, min_amplitude,
     max_amplitude, chosen_templates), filters in parameters:
    
    dir_name = util.directory()

    # make training data
    (x_detect, y_detect,
     x_triage, y_triage,
     x_ae, y_ae) = make.training_data(CONFIG,
                                      spike_train,
                                      chosen_templates,
                                      min_amplitude,
                                      max_amplitude,
                                      n_spikes,
                                      path_to_data)
    
    # detector
    detect_name = path.join(path_to_here, 'models',
                            'detect-'+dir_name+'.ckpt')

    _, waveform_length, n_neighbors =  x_detect.shape

    detector = NeuralNetDetector(detect_name, filters,
                                 waveform_length, n_neighbors,
                                 threshold=0.5,
                                 channel_index=CONFIG.channel_index,
                                 n_iter=n_iter)

    detector.fit(x_detect, y_detect)
    
    # triage
    triage_name = path.join(path_to_here, 'models',
                            'triage-'+dir_name+'.ckpt')

    _, waveform_length, n_neighbors = x_triage.shape

    triage = NeuralNetTriage(triage_name, filters,
                             waveform_length=waveform_length,
                             threshold=0.5,
                             n_neighbors=n_neighbors,
                             n_iter=n_iter)
    
    triage.fit(x_triage, y_triage)
    
    # save test sets
    _ = path.join(path_to_here, 'models', 'x-triage-'+dir_name+'.npy')
    np.save(_, x_triage)
    
    _ = path.join(path_to_here, 'models', 'y-triage-'+dir_name+'.npy')
    np.save(_, y_triage)
    
    _ = path.join(path_to_here, 'models', 'x-detect-'+dir_name+'.npy')
    np.save(_, x_detect)
    
    _ = path.join(path_to_here, 'models', 'y-detect-'+dir_name+'.npy')
    np.save(_, y_detect)

    
    print(detect_name, triage_name, n_spikes,
          min_amplitude, max_amplitude, filters)

100%|██████████| 3/3 [00:08<00:00,  2.92s/it]


AxisError: axis 2 is out of bounds for array of dimension 2